In [ ]:
%matplotlib inline 

import os
import sys
sys.path.append(os.path.abspath('../src/'))

import pandas as pd
import numpy as np
import datetime


import matplotlib.pyplot as plt
import seaborn as sns
sns.plotting_context('paper')

from wordGraph import *
from textDataset import *
from trainner import TrainnerNLP

from torchvision import transforms
import networkx as nx

from models import BoWClassifier, Tfid

import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
path2data = '../data/toxic/'

In [ ]:
text = {
    'train': TextProcessing(path2data + 'train/', is_train = True),
    'train': TextProcessing(path2data + 'train/', is_valid = True),
    'test':  TextProcessing(path2data + 'test/',  is_test=True),
}

In [ ]:
text['train'].data.head()

In [ ]:
text['train'].process_data(col = 'comment_text', lemmalize = False, stem = False)

In [ ]:
text['train'].data.head()

In [ ]:
text['test'].data.head()

In [ ]:
col = 'comment_text_data'

In [ ]:
# index into the Bag of words vector
words_ix = {}
words_voc = []
for index, row in text['train'].data[col].iteritems():
    for word in row.split():
        words_voc.append(word)
        if word not in words_ix:
            words_ix[word] = len(words_ix)

In [ ]:
VOCAB_SIZE = len(words_ix)
NUM_LABELS = 6
print('VOCAB_SIZE: {} NUM_LABELS: {}'.format(VOCAB_SIZE, NUM_LABELS))

In [ ]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

In [ ]:
loss_fn = nn.MultiLabelMarginLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
params = {'model' : model, 
    'criterion': loss_fn,  
    'optimizer': optimizer, 
}

In [ ]:
y_train = text['train'].data[['toxic', 'severe_toxic', 'obscene', 'threat',
   'insult', 'identity_hate']].values

x_train = np.vstack( text['train'].make_bow_vector('comment_text_data', words_ix))

In [ ]:
text['test'].process_data(col = 'comment_text', lemmalize = False, stem = False)
x_test = np.vstack( text['test'].make_bow_vector('comment_text_data', words_ix))

In [ ]:
dsets = {
            'train': TextDataset(x_train, y_train),
            'test': TextDataset(x_train, is_test = True)
}
    
dset_loaders = create_dataLoader(dsets, 10, pin_memory=False, use_shuffle= True)

In [ ]:
NLPtrainner = TrainnerNLP(params)

In [ ]:
NLPtrainner.train(dset_loaders['train'], 10)